In [1]:
import numpy as np
import pandas as pd
import dgl
import torch

In [2]:
live = pd.read_csv('./input/live.txt', header=None, sep=" ")
live.columns = ["country", "uid", "rid", "duration"]
live = live[live['duration']>=10]
print(live.shape, live.nunique()['uid'], live.nunique()['rid'])

(8652583, 4) 1295984 157067


In [5]:
friend = pd.read_csv('./input/friend.txt', header=None, sep=" ")
friend.columns = ["uid_from", "uid_to"]
print(friend.shape, friend.nunique()['uid_from'], friend.nunique()['uid_to'])

(50296888, 2) 4043024 4042968


In [6]:
user_ids = live['uid'].unique()
room_ids = live['rid'].unique()
countrys = live['country'].unique()
user_id_exist = dict(zip(user_ids, [1]*len(user_ids)))

friend['label_from'] = friend['uid_from'].map(user_id_exist)
friend['label_to'] = friend['uid_to'].map(user_id_exist)
friend = friend[~friend['label_from'].isnull() & ~friend['label_to'].isnull()]

In [7]:
user_id_map = dict(zip(user_ids, range(0, len(user_ids))))
room_id_map = dict(zip(room_ids, range(0, len(room_ids))))
country_map = dict(zip(countrys, range(0, len(countrys))))

live['uid_node'] = live['uid'].map(user_id_map)
live['rid_node'] = live['rid'].map(room_id_map)
friend['uid_from_node'] = friend['uid_from'].map(user_id_map)
friend['uid_to_node'] = friend['uid_to'].map(user_id_map)
live['country_feat'] = live['country'].map(country_map)

In [8]:
graph_data = {
    ('user', 'clicked', 'room'): (live['uid_node'].values, live['rid_node'].values),
    ('room', 'clicked-by', 'user'): (live['rid_node'].values, live['uid_node'].values),
    ('user', 'friend', 'user'): (friend['uid_from_node'].values, friend['uid_to_node'].values),
}
g = dgl.heterograph(graph_data, idtype=torch.int64)
g

In [27]:
g.nodes['user'].data['uid'] = torch.LongTensor(list(range(len(user_ids))))

user_country = live.groupby('uid_node')['country_feat'].apply(set).map(list).map(lambda x:x[0]).values
g.nodes['user'].data['country'] = torch.LongTensor(user_country)

neg_weights = live['uid_node'].value_counts().sort_index().map(lambda x:pow(x, 0.75)).values
g.nodes['user'].data['neg_weight'] = torch.FloatTensor(neg_weights)


In [35]:
g.nodes['user'].data

{'uid': tensor([      0,       1,       2,  ..., 1295981, 1295982, 1295983]), 'country': tensor([ 0,  1,  2,  ..., 27, 38, 38]), 'neg_weight': tensor([4.3035, 6.4474, 8.0000,  ..., 1.0000, 1.0000, 1.0000])}

In [1]:
from graph import *

In [2]:
g, user_ids = build_graph()
print(g)

params: duration_min 10
uid cnt 1295984 rid cnt 157067 country cnt 230
live cnt 8652583 friend cnt 11616526
Graph(num_nodes={'room': 157067, 'user': 1295984},
      num_edges={('room', 'clicked-by', 'user'): 8652583, ('user', 'clicked', 'room'): 8652583, ('user', 'friend', 'user'): 11616526},
      metagraph=[('room', 'user', 'clicked-by'), ('user', 'room', 'clicked'), ('user', 'user', 'friend')])


In [8]:
g.nodes['user'].data

{'uid': tensor([      0,       1,       2,  ..., 1295981, 1295982, 1295983]), 'country': tensor([ 0,  1,  2,  ..., 27, 38, 38]), 'neg_weight': tensor([4.3035, 6.4474, 8.0000,  ..., 1.0000, 1.0000, 1.0000])}

In [3]:
g.ndata.pop

defaultdict(<class 'dict'>, {'uid': {'user': tensor([      0,       1,       2,  ..., 1295981, 1295982, 1295983])}, 'country': {'user': tensor([ 0,  1,  2,  ..., 27, 38, 38])}, 'neg_weight': {'user': tensor([4.3035, 6.4474, 8.0000,  ..., 1.0000, 1.0000, 1.0000])}})